In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint

In [ ]:
xMin = 1e-6
xMax = 100 

# analytical bounds

## Gaussian profile

In [ ]:
root_rhoL = find_root(x*arctan(x)-1, 0.2, 4)
show("sqrt(rhoL)=", root_rhoL)

Delt = sqrt(1+root_rhoL**2)
show("Delta=", Delt)

# pi.n()/(rlr*Delt)
CLlarge = pi.n()/(Delt)
show("bound=", CLlarge)

In [ ]:
## analytical approximation
root_rhoL = (4/pi).n()
show("sqrt(rhoL)=", root_rhoL)

Delt = sqrt( 1+root_rhoL**2 )
show("Delta=", Delt)

# CLlargeA = (pi/sqrt(1+(4/pi)**2)).n()
CLlargeA = 2*arctan(root_rhoL) / Delt
show("bound=", CLlargeA)

In [ ]:
## lower bound
pltGL  = plot_loglog( CLlargeA, (x,2,xMax), color="darkblue", thickness=2, linestyle=":",
                    legend_label=" Eq. (20)")
pltGL += plot_loglog(2*x, (x,xMin,.2), color="blue", thickness=2,
                   legend_label=" Eq. (22)")


pltGL.show()

In [ ]:
# upper bound

# pltGU  = plot_loglog( 2*x * np.sqrt(-2*exp(1)*np.log(x)), (x,1e-4,1e-1), color="green", thickness=1)
# pltGU += plot_loglog( pi, (x,exp(-1),100), color="darkgreen", thickness=1)

# pltGU  = plot_loglog( 2*x * np.sqrt(-2*np.log(x)) * 2.3968950177775796, (x,1e-4,1e-1), color="green", thickness=1)
# pltGU += plot_loglog( 2*pi, (x,exp(-1),100), color="darkgreen", thickness=1)

pltGU  = plot_loglog( 2*pi, (x,exp(-1),xMax), color="darkgreen", thickness=2, linestyle=":",
                    legend_label=" Eq. (23)")
pltGU += plot_loglog( 4*x * np.sqrt(-exp(1)*np.log(x)), (x,xMin,1e-1), color="green", thickness=2,
                    legend_label=" Eq. (24)")

pltGU

In [ ]:
boundsG = pltGL+pltGU
boundsG.show()

# numerical evaluation of boundary

### setup common variables

In [ ]:
##  array with c-values
c_aux = np.linspace(-20,10,5000)
c = 2**c_aux

##  array with d-values
d_aux = np.linspace(-20,10,300)
d = 2**d_aux

## time array for ODE integration
tG = np.linspace(0, 20, 1+20*100)

##  initial condition for ODE
yI = [0]

### Gaussian profile

In [ ]:
@interact

def _( _d = slider(srange(.01,5,.1), default=0.5), ):
    
    dSquared = _d * _d
    gRate = 4 * np.log(1.+1./dSquared)

    pl  = plot( - dSquared + (1+dSquared) * np.exp( -x**2 * gRate ), (x,0,5), thickness=3 )
    pl += plot( -dSquared, (x,0,5), color="gray", thickness=3 )
    pl.show()
    return

In [ ]:
def gauss_integration (_c, _d) :

    ##  setup and solve ODE
    cSquared = _c * _c
    dSquared = _d * _d
    gRate = 4 * np.log(1.+1./dSquared)

    def ricc(z,t):
        dzdt = cSquared * z*z - dSquared + (1+dSquared) * np.exp( -t**2 * gRate )
        return dzdt

    # return solution
    return odeint(ricc,yI,tG)

In [ ]:
@interact

def _( _c = slider(srange(.01,5,.1), default=0.5),
       _d = slider(srange(.01,5,.1), default=0.5), ):
    sol = gauss_integration (_c, _d)
    
    yMin = - _d / _c
    yMax = - 2* yMin
    pl  = line( zip(tG,sol), thickness=5, ymin = yMin, ymax = yMax )
    pl.show()
    return

In [ ]:
## array with tipping values
c_Gauss = np.linspace(-20,10,300)

j    =  0   # index for  c-values
zMax = -1

for i in range (0,300) :          # index for  d-values
    ## increase  c-values   until  we hit a runaway trajectory
    while ( zMax <= d[i]/c[j] ) :       
        ##  setup and solve ODE
        sol = gauss_integration (c[j], d[i])
        
        ## assign asymptotic value to  zMax
        zMax = np.amax(sol)

        ##  increase index for  c-value
        j += 1

    ##  store latest c-value  in  c_tip
    j -= 1
    c_Gauss[i] = c[j]
    zMax = 0
    print( "i=", i, "\t d=", d[i], "\t C=", c[j] )


In [ ]:
ptGauss = line( zip(d, c_Gauss), thickness=4, color="red",
    axes_labels = [r"$d$", r"$c$"],
    )

#pt = plot_loglog( -10*x*log(np.abs(x)), (x,1e-5,1e3), ymax=10)
#ptL2 = plot_loglog( 50*x, (x,1e-5,1e3), ymax=10, color="pink")


(boundsG+ptGauss).show( scale="semilogx", xmin=10, xmax=100, ymin=1.9, ymax=2.1,
                      title = r"$c_\infty$ = %.4f"%c_Gauss[-1] )

# Figure 1

In [ ]:
plt_params = dict( scale="loglog", 
              xmin = xMin, 
              xmax = xMax, 
              ymin = 1e-6, 
              ymax = 100, 
                  ticks = [[1e-6,1e-4,1e-2,1,1e2], [1e-6,1e-4,1e-2,1,1e2]],
              frame = True,
                 )

In [ ]:
ptGauss.set_legend_options(font_size=8, loc=4)
boundsG.set_legend_options(font_size=8, loc=4)

ptGauss = line( zip(d, c_Gauss), thickness=2, color="red",
    axes_labels = [r"$d$", r"$c$"],
    legend_label = " numerical data"
    )
#ptGauss += points( zip(d, c_Gauss), color="green",
#    axes_labels = ["d", "c"],
#     xmax=3
#    )

#pt = plot_loglog( -10*x*log(np.abs(x)), (x,1e-5,1e3), ymax=10)
#ptL2 = plot_loglog( 50*x, (x,1e-5,1e3), ymax=10, color="pink")


(boundsG+ptGauss).show( gridlines = True, **plt_params )

In [ ]:
bbox = {'boxstyle':"square", 'fc':"white", 'ec':"white", 'lw':2}
ptG_text  = text("unsafe", (1e-4,   2), fontsize=10,  color="black",
                            bounding_box=bbox)
ptG_text += text("safe",  (0.001, 4e-5), fontsize=10, color="black",
                            bounding_box=bbox)
ptG_text.set_legend_options(font_size=7)


(ptG_text + boundsG+ptGauss).show( **plt_params )

In [ ]:
(ptG_text+ptGauss + boundsG).save_image("overshoot_Gauss_dc.pdf", transparent=True, 
               fontsize=8,                  # size of text font
              figsize=(3.6,2.4),                # figure size)
                                                  **plt_params )
(ptG_text+ptGauss + boundsG).save_image("overshoot_Gauss_dc.png", transparent=True, 
               fontsize=8,                  # size of text font
              figsize=(3.6,2.4),                # figure size)
                                                   **plt_params )